# constellation detection Model

In [ ]:
import os
import yaml
from ultralytics import YOLO
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
from tqdm import tqdm

# Configuration
DATASET_PATH = "stars_constellations_dataset"  # Path to dataset
IMG_SIZE = 640  # Target image size
BATCH_SIZE = 16
EPOCHS = 10
LEARNING_RATE = 0.001  # Changed to 0.001 as requested
MODEL_TYPE = "yolo11s.pt"  # Changed to YOLOv11s (small variant)

# Define constellation classes (16 classes as per requirements)
CLASSES = [
    "Aquila", "Bootes", "Canis Major", "Canis Minor", "Cassiopeia",
    "Cygnus", "Gemini", "Leo", "Lyra", "Moon", 
    "Orion", "Pleiades", "Sagittarius", "Taurus", "Ursa Major", "Moon"
]

def create_dataset_structure():
    """
    Create the necessary directory structure for YOLOv11 format
    """
    os.makedirs(f"{DATASET_PATH}/images/train", exist_ok=True)
    os.makedirs(f"{DATASET_PATH}/images/val", exist_ok=True)
    os.makedirs(f"{DATASET_PATH}/images/test", exist_ok=True)
    os.makedirs(f"{DATASET_PATH}/labels/train", exist_ok=True)
    os.makedirs(f"{DATASET_PATH}/labels/val", exist_ok=True)
    os.makedirs(f"{DATASET_PATH}/labels/test", exist_ok=True)
    
    return

def preprocess_images(input_dir, output_dir):
    """
    Preprocess images by resizing to 640x640 as per requirements
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    image_files = [f for f in os.listdir(input_dir) if f.endswith(('.jpg', '.png', '.jpeg'))]
    
    for img_file in tqdm(image_files, desc="Preprocessing images"):
        img_path = os.path.join(input_dir, img_file)
        img = cv2.imread(img_path)
        
        # Check if image loaded correctly
        if img is None:
            print(f"Failed to load image: {img_path}")
            continue
        
        # Resize to square 640x640
        img_resized = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        
        # Optional: Apply additional preprocessing specifically for star images
        # Increase contrast to make stars more visible
        img_enhanced = enhance_star_visibility(img_resized)
        
        # Save the preprocessed image
        output_path = os.path.join(output_dir, img_file)
        cv2.imwrite(output_path, img_enhanced)
    
    return

def enhance_star_visibility(image):
    """
    Apply image processing to enhance star visibility
    """
    # Convert to grayscale for brightness analysis
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) if len(image.shape) == 3 else image.copy()
    
    # Apply histogram equalization to enhance contrast
    equalized = cv2.equalizeHist(gray)
    
    # If original image was color, convert back to color
    if len(image.shape) == 3:
        # Create a 3-channel image from the equalized grayscale
        equalized_color = cv2.cvtColor(equalized, cv2.COLOR_GRAY2BGR)
        return equalized_color
    
    return equalized

def split_dataset(image_list, labels_list, train_ratio=0.7, val_ratio=0.2, test_ratio=0.1):
    """
    Split dataset into train, validation and test sets based on specified ratio
    """
    # First split into train and temporary sets
    train_images, temp_images, train_labels, temp_labels = train_test_split(
        image_list, labels_list, test_size=(val_ratio + test_ratio), random_state=42
    )
    
    # Split temporary set into validation and test sets
    val_ratio_adjusted = val_ratio / (val_ratio + test_ratio)
    val_images, test_images, val_labels, test_labels = train_test_split(
        temp_images, temp_labels, test_size=(1 - val_ratio_adjusted), random_state=42
    )
    
    return (train_images, train_labels), (val_images, val_labels), (test_images, test_labels)

def create_yaml_config():
    """
    Create YAML configuration file for YOLOv11 training
    """
    config = {
        'path': os.path.abspath(DATASET_PATH),
        'train': 'images/train',
        'val': 'images/val',
        'test': 'images/test',
        'names': {i: name for i, name in enumerate(CLASSES)}
    }
    
    with open(f"{DATASET_PATH}/constellation_data.yaml", 'w') as f:
        yaml.dump(config, f, default_flow_style=False)
    
    print(f"Created YAML configuration at {DATASET_PATH}/constellation_data.yaml")
    return f"{DATASET_PATH}/constellation_data.yaml"

def train_model(yaml_path):
    """
    Train the YOLOv11s model using the dataset with AdamW optimizer
    """
    # Load a pretrained YOLOv11s model
    model = YOLO(MODEL_TYPE)
    
    # Train the model with optimized hyperparameters for star detection
    # Note: YOLOv11 uses AdamW by default for most training scenarios
    results = model.train(
        data=yaml_path,
        epochs=EPOCHS,
        batch=BATCH_SIZE,
        imgsz=IMG_SIZE,
        optimizer="AdamW",  # Changed to AdamW optimizer
        lr0=LEARNING_RATE,  # Initial learning rate set to 0.001
        lrf=0.01,          # Final learning rate as a fraction of lr0
        momentum=0.937,    # SGD momentum/Adam beta1 (not used with AdamW)
        weight_decay=0.01,  # AdamW weight decay - increased for better regularization
        warmup_epochs=3.0,    # Warmup epochs for learning rate
        warmup_momentum=0.8,  # Warmup initial momentum
        box=7.5,           # Box loss gain (higher for better bounding box precision)
        cls=0.5,           # Class loss gain (lower for astronomical objects with less class variation)
        hsv_h=0.015,       # Image HSV-Hue augmentation (fraction)
        hsv_s=0.7,         # Image HSV-Saturation augmentation (fraction)
        hsv_v=0.4,         # Image HSV-Value augmentation (fraction) - important for brightness variations
        degrees=0.0,       # Image rotation (+/- deg) - minimal rotation for astronomical images
        translate=0.1,     # Image translation (+/- fraction)
        scale=0.5,         # Image scale (+/- gain) - helps with detection at different scales
        fliplr=0.5,        # Image flip left-right (probability)
        mosaic=1.0,        # Mosaic augmentation (probability)
        mixup=0.0,         # Mixup augmentation (probability) - disabled as it might confuse constellation patterns
        copy_paste=0.0,    # Segment copy-paste (probability) - not needed for star detection
        conf=0.25,         # Detection confidence threshold
        iou=0.7,           # Intersection over union (IoU) threshold for NMS
        max_det=300,       # Maximum detections per image
        name="constellation_detector_adamw",  # Updated name to reflect AdamW usage
        device=0,          # Use your GPU
        amp=True           # Enable Automatic Mixed Precision for better performance with AdamW
    )
    
    return model, results

def evaluate_model(model):
    """
    Evaluate model performance using mAP50 and other metrics
    """
    # Validate the model (this will calculate mAP50 as specified in requirements)
    results = model.val()
    
    # Safely convert metrics to float values
    def safe_float_convert(value):
        try:
            if hasattr(value, 'item'):  # Handle tensor values
                return value.item()
            elif hasattr(value, '__len__') and len(value) == 1:  # Handle length-1 arrays
                return float(value[0])
            else:
                return float(value)  # Try direct conversion
        except (TypeError, ValueError, IndexError):
            return 0.0  # Default value if conversion fails
    
    metrics = {
        "mAP50": safe_float_convert(results.box.map50),
        "precision": safe_float_convert(results.box.p),
        "recall": safe_float_convert(results.box.r),
        "f1": safe_float_convert(results.box.f1)
    }
    
    print("\nEvaluation Metrics:")
    print(f"• mAP50: {metrics['mAP50']:.3f}")
    print(f"• Precision: {metrics['precision']:.3f}")
    print(f"• Recall: {metrics['recall']:.3f}")
    print(f"• F1 Score: {metrics['f1']:.3f}")
    
    return metrics

def detect_constellations(model, image_path):
    """
    Perform constellation detection on a single image with optimized inference parameters
    """
    # Run inference with parameters optimized for star detection
    results = model.predict(
        image_path, 
        conf=0.25,       # Detection confidence threshold
        iou=0.7,         # NMS IoU threshold
        max_det=300,     # Maximum number of detections per image
        augment=True     # TTA (Test Time Augmentation)
    )
    
    # Process results
    result = results[0]
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Display results
    plt.figure(figsize=(10, 10))
    plt.imshow(result.plot())
    plt.axis('off')
    plt.title('Constellation Detection')
    plt.show()
    
    # Print detections
    for box in result.boxes:
        class_id = int(box.cls[0].item())
        confidence = box.conf[0].item()
        coordinates = box.xyxy[0].tolist()
        print(f"Detected {CLASSES[class_id]} with confidence {confidence:.2f} at {coordinates}")
    
    return result

def main():
    # Step 1: Create dataset structure
    create_dataset_structure()
    
    # Note: At this point you would need to:
    # 1. Collect/prepare your 1,750 labeled constellation images
    # 2. Convert annotations to YOLO format
    # 3. Place images and labels in the appropriate directories
    print("Please prepare your dataset by placing images and labels in the appropriate directories")
    
    # Step 2: Create YAML configuration
    yaml_path = create_yaml_config()
    
    # Step 3: Train the model with AdamW optimizer
    print("Training YOLOv11s model with AdamW optimizer (lr=0.001) for star constellation detection...")
    model, training_results = train_model(yaml_path)
    
    # Step 4: Evaluate the model
    print("Evaluating model performance...")
    metrics = evaluate_model(model)
    
    # Save the trained model
    model_save_path = "models/constellation_detector_yolo11s_adamw.pt"
    os.makedirs(os.path.dirname(model_save_path), exist_ok=True)
    model.export(format="torchscript")  # Export to PyTorch format
    print(f"Model saved to {model_save_path}")
    
    # For demonstration, you would use your own test image
    # detect_constellations(model, "path_to_test_image.jpg")
    
    print("Constellation detection model training and evaluation complete!")

if __name__ == "__main__":
    main()

Please prepare your dataset by placing images and labels in the appropriate directories
Created YAML configuration at stars_constellations_dataset/constellation_data.yaml
Training YOLOv11s model with AdamW optimizer (lr=0.001) for star constellation detection...
Ultralytics 8.3.133  Python-3.13.3 torch-2.7.0+cu118 CUDA:0 (NVIDIA GeForce GTX 1070, 8192MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=0.25, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=stars_constellations_dataset/constellation_data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=Non

train: Scanning C:\Users\Dell\Downloads\Star Constellation (2)\Star Constellation\Star Constellation\stars_constellations_dataset\labels\train.cache... 1530 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1530/1530 [00:00<?, ?it/s]


val: Fast image access  (ping: 0.10.0 ms, read: 151.769.0 MB/s, size: 48.0 KB)


val: Scanning C:\Users\Dell\Downloads\Star Constellation (2)\Star Constellation\Star Constellation\stars_constellations_dataset\labels\val.cache... 14 images, 0 backgrounds, 0 corrupt: 100%|██████████| 14/14 [00:00<?, ?it/s]


Plotting labels to runs\detect\constellation_detector_adamw\labels.jpg... 
optimizer: AdamW(lr=0.001, momentum=0.937) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.01), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\constellation_detector_adamw
Starting training for 10 epochs...
Closing dataloader mosaic

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...


10 epochs completed in 0.154 hours.
Optimizer stripped from runs\detect\constellation_detector_adamw\weights\last.pt, 19.2MB
Optimizer stripped from runs\detect\constellation_detector_adamw\weights\best.pt, 19.2MB

Validating runs\detect\constellation_detector_adamw\weights\best.pt...
Ultralytics 8.3.133  Python-3.13.3 torch-2.7.0+cu118 CUDA:0 (NVIDIA GeForce GTX 1070, 8192MiB)
YOLO11s summary (fused): 100 layers, 9,418,992 parameters, 0 gradients, 21.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  5.97it/s]


                   all         14         75      0.916      0.977      0.975      0.564
                Bootes          1          1      0.333          1      0.995      0.597
           Canis Major          9          9          1          1      0.995        0.7
           Canis Minor          9          9      0.889      0.889      0.852      0.325
            Cassiopeia          5          5          1          1      0.995      0.487
                Gemini          9          9          1          1      0.995       0.68
                   Leo          9          9          1          1      0.995      0.658
                  Moon          3          3          1          1      0.995       0.52
                 Orion          9          9          1          1      0.995      0.637
              Pleiades          7          7      0.857      0.857      0.918      0.375
            Ursa Major          9          9          1          1      0.995      0.572
                  Moo

val: Scanning C:\Users\Dell\Downloads\Star Constellation (2)\Star Constellation\Star Constellation\stars_constellations_dataset\labels\val.cache... 14 images, 0 backgrounds, 0 corrupt: 100%|██████████| 14/14 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:03<00:00,  3.19s/it]


                   all         14         75      0.929      0.977      0.976      0.563
                Bootes          1          1      0.333          1      0.995      0.597
           Canis Major          9          9          1          1      0.995        0.7
           Canis Minor          9          9      0.889      0.889      0.852      0.316
            Cassiopeia          5          5          1          1      0.995      0.487
                Gemini          9          9          1          1      0.995       0.68
                   Leo          9          9          1          1      0.995      0.658
                  Moon          3          3          1          1      0.995       0.52
                 Orion          9          9          1          1      0.995      0.637
              Pleiades          7          7          1      0.857      0.928      0.377
            Ursa Major          9          9          1          1      0.995      0.572
                  Moo

testing cuda installation

In [1]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda if torch.cuda.is_available() else 'N/A'}")
if torch.cuda.is_available():
    print(f"GPU device count: {torch.cuda.device_count()}")
    print(f"Current device: {torch.cuda.current_device()}")
    print(f"Device name: {torch.cuda.get_device_name()}")

print(f"PyTorch version: {torch.__version__}")
print(f"Built with CUDA: {torch.version.cuda}")

PyTorch version: 2.7.0+cu118
CUDA available: True
CUDA version: 11.8
GPU device count: 1
Current device: 0
Device name: NVIDIA GeForce GTX 1070
PyTorch version: 2.7.0+cu118
Built with CUDA: 11.8
